In [3]:
#!/usr/bin/env python

#Python Imports

from __future__ import print_function
import sys
import numpy as np
import math

#ROS Imports
from ackermann_msgs.msg import AckermannDriveStamped, AckermannDrive
import rospy
from sensor_msgs.msg import Image, LaserScan
from std_msgs import Float32, Float64

#PID CONTROL PARAMS
kp = 1.00 #TODO
kd = 0.001 #TODO
ki = 0.005 #TODO

#WALL FOLLOW PARAMS
servo_offset = 0.0
ANGLE_RANGE = 270 # Hokuyo 10LX has 270 degrees scan
DESIRED_DISTANCE_RIGHT = 0.9 # meters
DESIRED_DISTANCE_LEFT = 1.2
CAR_LENGTH = 0.50 # Traxxas Rally is 20 inches or 0.5 meters


#global variables
inda=0
indb=0
angle_a=0
angle_b=0
range_a=0
range_b=0

prev_error = 0.0
error = 0.0
integral = 0.0

VELOCITY = 2.00 # meters per second


ModuleNotFoundError: ignored

In [ ]:
class WallFollow:
    """ Implement Wall Following on the car
    """
    def __init__(self):
        #Topics & Subs, Pubs
        lidarscan_topic = '/scan'
        drive_topic = '/nav'

        self.lidar_sub = rospy.Subscriber(lidarscan_topic,LaserScan,self.lidar_callback,queue_size=1000)
        self.drive_pub = rospy.Publisher(drive_topic,AckermannDriveStamped,queue_size=1000)

    def getRange(self, data, angle):
        # data: single message from topic /scan
        # angle: between -45 to 225 degrees, where 0 degrees is directly to the right
        # Outputs length in meters to object with angle in lidar scan field of view
        #make sure to take care of nans etc.
        #TODO: implement
        global indb
        global inda
        indb=np.floor((90.0))

        global b_angle
        b_angle=180
        global theta
        theta=45
        global a_angle
        a_angle=b_angle - theta
        global alpha
        global a
        global b
        
        alpha=math.atan((a*math.cos(theta)-b)/(a*math.sin(theta)))
        for i in range(data):
          if not np.isnan(data[i]) or not np.isinf(data[i]):
             indb = i if angle == data.angle_increment*(180-45+alpha) else indb
             b=data[indb]
             inda = i if angle == data.angle_increment*(180-45+alpha+theta) else inda
             a=data[inda]
        global dt
        global dt_1
        dt=b*math.cos(alpha)
        dt_1=dt+CAR_LENGTH
        self.followLeft(data,dt)
        return 0.0

    def pid_control(self, error, velocity):
        global integral
        global prev_error
        global kp
        global ki
        global kd
        angle = 0.0
        #TODO: Use kp, ki & kd to implement a PID controller for 
        global der_error
        der_error=error-prev_error
        global dtime
        dtime=rospy.get_time()
        integral+=error*dtime
        angle=-(kp*error+kd*(der_error/dtime)+ki*integral)
        dtime=rospy.get_time()-dtime
        prev_error=error

        if 0 <= angle <=10 :
          velocity = 1.5
        elif 10 < angle <=20 :
          velocity = 1.0
        else
          velocity = 0.5
        drive_msg = AckermannDriveStamped()
        drive_msg.header.stamp = rospy.Time.now()
        drive_msg.header.frame_id = "laser"
        drive_msg.drive.steering_angle = angle
        drive_msg.drive.speed = velocity
        self.drive_pub.publish(drive_msg)

    def followLeft(self, data, leftDist):
        #Follow left wall as per the algorithm 
        #TODO:implement
        error=DESIRED_DISTANCE_LEFT-leftDist
        return error 

    def lidar_callback(self, data):
        """ 
        """
        error = self.followLeft #TODO: replace with error returned by followLeft
        #send error to pid_control
        self.pid_control(error, VELOCITY)


SyntaxError: ignored

In [ ]:
def main(args):
    rospy.init_node("WallFollow_node", anonymous=True)
    wf = WallFollow()
    rospy.sleep(0.1)
    rospy.spin()

if __name__=='__main__':
	main(sys.argv)